# 데이터 불러오기 및 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

aisles = pd.read_csv('/content/drive/MyDrive/AIB/cp1/aisles.csv')
departments = pd.read_csv('/content/drive/MyDrive/AIB/cp1/departments.csv')
order_products_prior = pd.read_csv('/content/drive/MyDrive/AIB/cp1/order_products__prior.csv')
order_products_train = pd.read_csv('/content/drive/MyDrive/AIB/cp1/order_products__train.csv')
orders = pd.read_csv('/content/drive/MyDrive/AIB/cp1/orders.csv')
products = pd.read_csv('/content/drive/MyDrive/AIB/cp1/products.csv')

**orders(len = 3421083)**  
- order_id = 주문번호
- user_id = 주문을 한 유저번호
- eval_set = prior/train/test 중 하나 각각에 따라 order_products_prior/order_products_train으로 연결 (eval_set이 test인 경우는 고민해봐야할듯)
- order_number = 이 유저가 이번이 몇번째 주문인지?
- order_hour_of_day = 하루 중 어느 시간대에 주문을 했는지?
- day_since_prior_order = 지난 마지막 주문으로부터 현재 주문까지 그 차이가 얼마나 되는지(NaN인 경우는 order_number가 1, 즉 첫 주문인 경우)

In [ ]:
orders.head()

order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0

day_since_prior_order에서 결측치는 첫 주문, 즉 이전 주문과의 차이가 없으므로 모두 0으로 대체한다

In [ ]:
orders['days_since_prior_order'] = orders['days_since_prior_order'].fillna(0)

orders.head()

order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     0.0  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0

**order_products_prior/train(len = 32434489/1384617)**  
- add_to_cart_order = 해당 주문건에 대해서 장바구니에 실린 순서
- reordered = 이전에 주문한 적이 있는 품목이면 1, 아니면 0

In [ ]:
order_products_prior.head()

order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0

In [ ]:
order_products_train.head()

order_id  product_id  add_to_cart_order  reordered
0         1       49302                  1          1
1         1       11109                  2          1
2         1       10246                  3          0
3         1       49683                  4          0
4         1       43633                  5          1

**products(총 49688개의 제품)**  
- aisle = 세부 카테고리(ex.aisle_id = 61 = cookie cake)
- department = 카테고리(ex. department_id = 19 = snack)

In [ ]:
products.head()

product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id  
0             19  
1             13  
2              7  
3              1  
4             13

**aisle(세부 카테고리)**  
총 134개

In [ ]:
aisles.head()

aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation

**departmnent(카테고리)**  
frozen, bakery, alcohol 등 총 21개 카테고리

In [ ]:
departments.head()

department_id department
0              1     frozen
1              2      other
2              3     bakery
3              4    produce
4              5    alcohol

# Feature Engineering - orders와 order_products_prior을 병합

In [ ]:
prior = pd.merge(orders, order_products_prior, on = 'order_id', how = 'inner')
prior.head()

order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2539329        1    prior             1          2                  8   
2   2539329        1    prior             1          2                  8   
3   2539329        1    prior             1          2                  8   
4   2539329        1    prior             1          2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  
0                     0.0         196                  1          0  
1                     0.0       14084                  2          0  
2                     0.0       12427                  3          0  
3                     0.0       26088                  4          0  
4                     0.0       26405                  5          0

In [ ]:
# prior.to_csv('/content/drive/MyDrive/AIB/cp1/prior.csv', index = False)

# User에 관한 Feature Engineering(user_id 사용)

**만들 수 있는 변수들**  
- 고객당 총 주문 수
- 고객이 한 주문당 담은 평균 제품 수
- 고객이 가장 주문을 많이 한 요일
- 고객이 가장 주문을 많이 한 시간대
- 고객의 재주문율
- 고객이 이전 주문과 현재 주문 사이의 간격
- 고객이 주문한 총 제품의 수

고객당 총 주문 수

In [ ]:
u_order_num = prior.groupby('user_id')['order_number'].max().to_frame('u_order_num').reset_index()
u_order_num.head()

user_id  u_order_num
0        1           10
1        2           14
2        3           12
3        4            5
4        5            4

고객이 한 주문당 담은 평균 제품 수

In [ ]:
basket_amount = prior.groupby(['user_id', 'order_id'])['product_id'].count().to_frame('basket_amount').reset_index()

u_avg_basket = basket_amount.groupby('user_id')['basket_amount'].mean().to_frame('u_avg_basket').reset_index()

u_avg_basket.head()

user_id  u_avg_basket
0        1      5.900000
1        2     13.928571
2        3      7.333333
3        4      3.600000
4        5      9.250000

고객이 가장 주문을 많이 한 요일

In [ ]:
from scipy import stats

u_most_dow = prior.groupby('user_id')['order_dow'].aggregate(lambda x : stats.mode(x)[0]).to_frame('u_most_dow').reset_index()
u_most_dow.head()

user_id  u_most_dow
0        1           4
1        2           2
2        3           0
3        4           4
4        5           3

고객이 가장 주문을 많이 한 시간대

In [ ]:
u_most_hod = prior.groupby('user_id')['order_hour_of_day'].aggregate(lambda x : stats.mode(x)[0]).to_frame('u_most_hod').reset_index()
u_most_hod.head()

user_id  u_most_hod
0        1           7
1        2           9
2        3          16
3        4          15
4        5          18

고객의 재주문율

In [ ]:
u_reorder = prior.groupby('user_id')['reordered'].mean().to_frame('u_reorder').reset_index()
u_reorder.head()

user_id  u_reorder
0        1   0.694915
1        2   0.476923
2        3   0.625000
3        4   0.055556
4        5   0.378378

고객이 이전 주문과 현재 주문 사이의 간격

In [ ]:
u_avg_gap = prior.groupby('user_id')['days_since_prior_order'].mean().to_frame('u_avg_gap').reset_index()
u_avg_gap.head()

user_id  u_avg_gap
0        1  18.542373
1        2  14.902564
2        3  10.181818
3        4  11.944444
4        5  10.189189

고객이 주문한 총 제품의 수

In [ ]:
u_product_num = prior.groupby('user_id').size().to_frame('u_product_num').reset_index()
u_product_num.head()

user_id  u_product_num
0        1             59
1        2            195
2        3             88
3        4             18
4        5             37

**user에 대해 만든 피쳐들을 합쳐서 user_info라는 데이터프레임 생성**
- 고객당 총 주문 수(u_order_num)
- 고객이 한 주문당 담은 평균 제품 수(u_avg_basket)
- 고객이 가장 주문을 많이 한 요일(u_most_dow)
- 고객이 가장 주문을 많이 한 시간대(u_most_hod)
- 고객의 재주문율(u_reorder)
- 고객이 이전 주문과 현재 주문 사이의 간격(u_avg_gap)
- 고객이 주문한 총 제품의 수(u_product_num)

In [ ]:
user_info = u_order_num.merge(u_avg_basket, on = 'user_id', how = 'left')
user_info = user_info.merge(u_most_dow, on = 'user_id', how = 'left')
user_info = user_info.merge(u_most_hod, on = 'user_id', how = 'left')
user_info = user_info.merge(u_reorder, on = 'user_id', how = 'left')
user_info = user_info.merge(u_avg_gap, on = 'user_id', how = 'left')
user_info = user_info.merge(u_product_num, on = 'user_id', how = 'left')

user_info.head()

user_id  u_order_num  u_avg_basket  u_most_dow  u_most_hod  u_reorder  \
0        1           10      5.900000           4           7   0.694915   
1        2           14     13.928571           2           9   0.476923   
2        3           12      7.333333           0          16   0.625000   
3        4            5      3.600000           4          15   0.055556   
4        5            4      9.250000           3          18   0.378378   

   u_avg_gap  u_product_num  
0  18.542373             59  
1  14.902564            195  
2  10.181818             88  
3  11.944444             18  
4  10.189189             37

In [ ]:
# user_info.to_csv('/content/drive/MyDrive/AIB/cp1/user_info.csv', index = False)

# Product에 대한 Feature Engineering(product_id 사용)

**만들 수 있는 변수들**  
- 제품의 주문횟수
- 제품의 재주문율
- 제품이 각 주문에서 장바구니에 담긴 평균 순서

제품의 주문횟수

In [ ]:
p_order_num = prior.groupby('product_id')['order_id'].count().to_frame('p_order_num').reset_index()
p_order_num.head()

product_id  p_order_num
0           1         1852
1           2           90
2           3          277
3           4          329
4           5           15

제품의 재주문율

In [ ]:
p_reorder = prior.groupby('product_id')['reordered'].mean().to_frame('p_reorder').reset_index()
p_reorder.head()

product_id  p_reorder
0           1   0.613391
1           2   0.133333
2           3   0.732852
3           4   0.446809
4           5   0.600000

제품이 각 주문에서 장바구니에 담긴 평균 순서

In [ ]:
p_avg_basketposition = prior.groupby('product_id')['add_to_cart_order'].mean().to_frame('p_avg_basketpositon').reset_index()
p_avg_basketposition.head()

product_id  p_avg_basketpositon
0           1             5.801836
1           2             9.888889
2           3             6.415162
3           4             9.507599
4           5             6.466667

**product에 대해 만든 피쳐들을 합쳐서 prod_info라는 데이터프레임 생성**  
- 제품의 주문횟수(p_order_num)
- 제품의 재주문율(p_reorder)
- 제품이 각 주문에서 장바구니에 담긴 평균 순서(p_avg_basketpostion)

In [ ]:
prod_info = p_order_num.merge(p_reorder, on = 'product_id', how = 'left')
prod_info = prod_info.merge(p_avg_basketposition, on = 'product_id', how = 'left')

prod_info.head()

product_id  p_order_num  p_reorder  p_avg_basketpositon
0           1         1852   0.613391             5.801836
1           2           90   0.133333             9.888889
2           3          277   0.732852             6.415162
3           4          329   0.446809             9.507599
4           5           15   0.600000             6.466667

In [ ]:
# prod_info.isnull().to_csv('/content/drive/MyDrive/AIB/cp1/prod_info.csv', index = False)

# User와 Product에 대한 Feature Engineering(user_id, product_id 사용)

**만들 수 있는 변수들**  
- 고객이 해당 제품을 주문한 횟수
- 고객이 해당 제품을 재주문할 확률
- 고객이 해당 제품을 장바구니에 담은 평균 순서

고객이 해당 제품을 주문한 횟수

In [ ]:
up_order_num = prior.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('up_order_num').reset_index()
up_order_num.head()

user_id  product_id  up_order_num
0        1         196            10
1        1       10258             9
2        1       10326             1
3        1       12427            10
4        1       13032             3

고객이 해당 제품을 재주문할 확률

In [ ]:
up_reorder = prior.groupby(['user_id', 'product_id'])['reordered'].mean().to_frame('up_reorder').reset_index()
up_reorder.head()

user_id  product_id  up_reorder
0        1         196    0.900000
1        1       10258    0.888889
2        1       10326    0.000000
3        1       12427    0.900000
4        1       13032    0.666667

고객이 해당 제품을 장바구니에 담은 평균 순서

In [ ]:
up_avg_basketposition = prior.groupby(['user_id', 'product_id'])['add_to_cart_order'].mean().to_frame('up_avg_basketposition').reset_index()
up_avg_basketposition.head()

user_id  product_id  up_avg_basketposition
0        1         196               1.400000
1        1       10258               3.333333
2        1       10326               5.000000
3        1       12427               3.300000
4        1       13032               6.333333

**user와 product에 대해 만든 피쳐들을 합쳐서 user_prod_info라는 데이터프레임 생성**  
- 고객이 해당 제품을 주문한 횟수(up_order_num)
- 고객이 해당 제품을 재주문할 확률(up_reorder)
- 고객이 해당 제품을 장바구니에 담은 평균 순서(up_avg_basketposition)

In [ ]:
user_prod_info = up_order_num.merge(up_reorder, on = ['user_id', 'product_id'], how = 'left')
user_prod_info = user_prod_info.merge(up_avg_basketposition, on = ['user_id', 'product_id'], how = 'left')

user_prod_info.head()

user_id  product_id  up_order_num  up_reorder  up_avg_basketposition
0        1         196            10    0.900000               1.400000
1        1       10258             9    0.888889               3.333333
2        1       10326             1    0.000000               5.000000
3        1       12427            10    0.900000               3.300000
4        1       13032             3    0.666667               6.333333

이전에 만든 user_info와 prod_info도 user_prod_info에 병합

In [ ]:
user_prod_info = user_prod_info.merge(user_info, on = 'user_id', how = 'left')
user_prod_info = user_prod_info.merge(prod_info, on = 'product_id', how = 'left')

user_prod_info.head()

user_id  product_id  up_order_num  up_reorder  up_avg_basketposition  \
0        1         196            10    0.900000               1.400000   
1        1       10258             9    0.888889               3.333333   
2        1       10326             1    0.000000               5.000000   
3        1       12427            10    0.900000               3.300000   
4        1       13032             3    0.666667               6.333333   

   u_order_num  u_avg_basket  u_most_dow  u_most_hod  u_reorder  u_avg_gap  \
0           10           5.9           4           7   0.694915  18.542373   
1           10           5.9           4           7   0.694915  18.542373   
2           10           5.9           4           7   0.694915  18.542373   
3           10           5.9           4           7   0.694915  18.542373   
4           10           5.9           4           7   0.694915  18.542373   

   u_product_num  p_order_num  p_reorder  p_avg_basketpositon  
0             59        35791   0.776480             3.721774  
1             59         1946   0.713772             4.277492  
2             59         5526   0.652009             4.191097  
3             59         6476   0.740735             4.760037  
4             59         3751   0.657158             5.622767

In [ ]:
user_prod_info.to_csv('/content/drive/MyDrive/AIB/cp1/user_prod_info.csv', index = False)